This file implements a time-series forecasting model using Prophet to predict trends and volumes for inpatient diagnoses. The project focuses on multi-series forecasting, where each primary diagnosis is predicted separately, and the results are then analyzed as a whole.

In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import itertools
import matplotlib.pyplot as plt

Data Preparation

In [ ]:
# Prepare historical data for each diagnosis on a monthly basis
df_rawatinap['tanggal_admisi'] = pd.to_datetime(df_rawatinap['tanggal_admisi'])
df_rawatinap['bulan_admisi'] = df_rawatinap['tanggal_admisi'].dt.to_period('M').dt.to_timestamp()

# Count the number of patients per diagnosis per month
df_monthly_diagnoses = df_rawatinap.groupby(
    ['bulan_admisi', 'icd_10', 'arti_icd']
).size().reset_index(name='jumlah_pasien').rename(
    columns={'bulan_admisi': 'ds', 'jumlah_pasien': 'y'}
)

# Filter for diagnoses that have at least 12 months of historical data
icd_bulan_count = df_monthly_diagnoses.groupby('icd_10')['ds'].nunique()
selected_icd = icd_bulan_count[icd_bulan_count >= 12].index
df_monthly_diagnoses = df_monthly_diagnoses[df_monthly_diagnoses['icd_10'].isin(selected_icd)]

Forecasting Model Implementation (Per Diagnosis)

In [ ]:
def run_diagnosis_forecast(df_input):
    """
    Performs forecasting for each eligible diagnosis.
    This function trains a Prophet model, predicts for the next 6 months,
    and evaluates the model's performance.
    """
    list_forecast = []
    all_metrics = []

    for icd_code in df_input['icd_10'].unique():
        df_subset = df_input[df_input['icd_10'] == icd_code][['ds', 'y']]

        model = Prophet()
        model.fit(df_subset)
        
        future = model.make_future_dataframe(periods=6, freq='MS')
        forecast = model.predict(future)
        
        # --- Model Evaluation ---
        forecast_eval = forecast[['ds', 'yhat']].merge(df_subset, on='ds', how='left').dropna()
        y_true = forecast_eval['y']
        y_pred = forecast_eval['yhat']

        mae = mean_absolute_error(y_true, y_pred)
        rmse = mean_squared_error(y_true, y_pred, squared=False)
        r2 = r2_score(y_true, y_pred)

        all_metrics.append({'icd_10': icd_code, 'MAE': mae, 'RMSE': rmse, 'R2': r2})

        # --- Store Results ---
        forecast['icd_10'] = icd_code
        forecast['arti_icd'] = df_input[df_input['icd_10'] == icd_code]['arti_icd'].iloc[0]
        list_forecast.append(forecast[['ds', 'yhat', 'icd_10', 'arti_icd']])
    
    # Combine all forecast and evaluation results
    df_forecast_all = pd.concat(list_forecast, ignore_index=True)
    df_metrics = pd.DataFrame(all_metrics)

    return df_forecast_all, df_metrics

# Execute the forecasting function
df_forecast, df_metrics = run_diagnosis_forecast(df_monthly_diagnoses)

# Print average evaluation results
print("=== Average Evaluation for All Diagnoses ===")
print(f"Average MAE: {df_metrics['MAE'].mean():.2f}")
print(f"Average RMSE: {df_metrics['RMSE'].mean():.2f}")
print(f"Average R²: {df_metrics['R2'].mean():.4f}")

Evaluation & Visualization

In [ ]:
# --- Display Top 5 Predicted Diagnoses ---
# Filter predicted data for future months
df_predicted = df_forecast[df_forecast['ds'] >= pd.to_datetime('2024-11-01')]
df_top5_diagnoses = df_predicted.sort_values(by=['ds', 'yhat'], ascending=[True, False]).groupby('ds').head(5)

print("\nTop-5 Predicted Diagnoses per Month:")
print(df_top5_diagnoses.to_string(index=False))

# --- Visualization ---
# Find the model with the highest R-squared from all diagnoses
best_model_icd = df_metrics.loc[df_metrics['R2'].idxmax()]
best_icd_code = best_model_icd['icd_10']

# Get the actual and forecasted data for the best diagnosis
df_actual_best = df_monthly_diagnoses[df_monthly_diagnoses['icd_10'] == best_icd_code].copy()
df_forecast_best = df_forecast[df_forecast['icd_10'] == best_icd_code].copy()

# Visualize the forecast
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(df_actual_best['ds'], df_actual_best['y'], label='Actual Data', color='blue', marker='o')
ax.plot(df_forecast_best['ds'], df_forecast_best['yhat'], label='Prediction', color='orange', linestyle='--', marker='o')

# Add a vertical line to mark the start of the prediction period
ax.axvline(x=df_actual_best['ds'].max(), color='gray', linestyle='--', linewidth=2, label='Start of Prediction Period')

ax.set_title(f'Diagnosis Forecast: {best_icd_code} (R²: {best_model_icd["R2"]:.2f})', fontsize=16)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Number of Patients', fontsize=12)
ax.legend()
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()